In [ ]:
!pip install gensim

In [ ]:
from gensim.models import Word2Vec
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords

In [ ]:
df = pd.read_json('/content/Cell_Phones_and_Accessories_5.json',lines=True)

df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


In [ ]:
df['reviewText'][0]

"They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again"

In [ ]:
df.shape

(194439, 9)

#Text preprocessing

1.   Lowercase

2. Remove punctuation and digits

3. Tokenize (split into words)

4. Optionally remove stopwords (e.g. “the”, “is”, “a”)



In [ ]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def clean_text(text):
  text = text.lower()
  text = re.sub(r'[^a-z]\s','',text)
  tokens = text.split()
  tokens = [word for word in tokens if word not in stop_words]
  return tokens

sentences = df['reviewText'].dropna().apply(clean_text).tolist()

In [ ]:
sentences[:1]

[['look',
  'good',
  'stick',
  'goodi',
  'like',
  'rounded',
  'shape',
  'always',
  'bumping',
  'siri',
  'kept',
  'popping',
  'irritatingi',
  'buy',
  'product',
  'like']]

In [ ]:
w2v_model = Word2Vec(
    sentences=sentences,
    vector_size=20,   # size of embedding (50–300 common)
    window=5,          # context window size
    min_count=5,       # ignore rare words
    workers=4,         # parallel threads
    sg=1,              # 1 = skip-gram, 0 = CBOW
    epochs=5
)

In [ ]:
print(w2v_model.wv['phone'])  # vector for "phone"


[-0.41984528  0.18943849  0.21109849  0.44347557 -0.04586567 -0.23933734
  0.23802352  0.8012135  -0.52146196  0.3315196   0.7226423   0.1490173
 -0.24983168 -0.07945525  0.6793828   0.31443012  0.01731009  0.25263232
 -0.528025   -0.42599252]


In [ ]:
w2v_model.wv.most_similar('phone')

[('phoneand', 0.9536722302436829),
 ('phoneeven', 0.9353592991828918),
 ('phonecan', 0.919044554233551),
 ('phonein', 0.9138513207435608),
 ('phoneit', 0.9094581007957458),
 ('phonewithout', 0.9069598317146301),
 ('phoneas', 0.9063229560852051),
 ('constantlythe', 0.905770480632782),
 ('tumbles', 0.9056392312049866),
 ('phonethis', 0.9018598794937134)]

In [ ]:
w2v_model.wv.similarity(w1='phone',w2='mobile')

0.5917026

In [ ]:
w2v_model.wv.similarity(w1='good',w2='pretty')

0.87854224